### Questions to ask

#### General questions

1) What past research has been done on these companies? Is there a valid source with its history towards slavery? Can we make a metric out of this?

2) Which businesses have a higher risk of slavery exploitation? Look at specific industry and as well locations

#### Relevant to the slavery statments

1) What does the slavery statement focus on?

- Can do something simple and get TF-IDFs for each document. This will give the frequency of word counts for each document and how that relates to the other documents. Gives a broader picture of what the document is about. If a document deviates from the norm, this could raise questions. 

2) How effectively do they respond to the standard and practices required? Is the information relevant?

- Can use the above to also answer this question.

3) Do they focus on future goals or things that they've already done?

- Can use a simple regex script to find tense. Can also use Named Entity Recognition to find more complex associations.

### The code at the bottom processes the PDFs given and saves it as a JSON file. The Keys are the titles in the documents and the values are the words between the titles. Can be saved in a NoSQL database.



### Test method 1. Using OCR to parse a pdf file.

In [184]:
import pytesseract
from wand.image import Image
import os

In [187]:
pdf_file = 'Data/coke_1.pdf'

In [ ]:
## Can open the file with 
# import urllib
# url = 'www.---.pdf'
# user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
# headers={'User-Agent':user_agent,} 
# request = urllib.request.Request(url, None, headers)
# response = urllib.request.urlopen(request)

#Chose to download them instead

In [178]:
%%time
#Converts all the pdf pages to pngs. Need this input (files) for pytesseract
files = []
with(Image(filename=pdf_file, resolution = 500)) as conn: 
    for index, image in enumerate(conn.sequence):
        image_name = os.path.splitext(pdf_file)[0] + str(index + 1) + '.png'
        Image(image).save(filename = image_name)
        files.append(image_name)

CPU times: user 1min 10s, sys: 22.2 s, total: 1min 32s
Wall time: 1min 34s


In [179]:
#wand image doesn't recognise PNG. Need to debug, using PIL as placeholder
from PIL import Image

In [180]:
#Reading all the pngs created with pytesseract
%%time
all_text = [pytesseract.image_to_string(Image.open(file)) for file in files]

CPU times: user 4.99 s, sys: 377 ms, total: 5.37 s
Wall time: 1min 13s


In [190]:
#Example of first page
print(all_text[0])

COU oly

EUROPEAN PARTNERS

Modern Slavery Statement 2018

This statement has been published in accordance with the Modern Slavery Act 2015.
It sets out the steps taken by Coca-Cola European Partners plc and its group
companies (CCEP), in particular Coca-Cola European Partners Great Britain Limited
and Coca-Cola European Partners Services Europe Limited, during the year ending 31
December 2018 to prevent modern slavery and human trafficking in our business and

supply chain.

Coca-Cola European
Partners is one of Europe's

LEADING CONSUMER
GOODS COMPANIES.

We sell, make and
distribute drinks to over

300m

CONSUMERS

13

COUNTRIES IN
WESTERN EUROPE

— APPROXIMATELY —

23,300

EMPLOYEES

aT

SUPPLIERS ACROSS
OUR MARKETS

2. Dbn

UNIT CASES SOLD
IN 2018

Introduction

We consider human and workplace rights — as
articulated in the United Nations’ Universal
Declaration of Human Rights, and the International
Labour Organization's Declaration on Fundamental
Principles and Rights at Work — t

#### Comments
- Gives more detailed answers but would take too long for more PDFs
- Not recommended, took about 3 minutes for one PDF, but can be optimized further and completed on a better CPU (AWS/GCP). Can be useful.

### Test method 2: Parsing the pdf file directly


#### Trying different libraries
##### - textract

In [ ]:
import textract

In [199]:
%%time
text = textract.process('coke_1.pdf', encoding='ascii')
text = str(text)
text = text.replace('\\n', ' ')

CPU times: user 146 ms, sys: 10.5 ms, total: 156 ms
Wall time: 253 ms


In [549]:
text[:1000]

'b"Modern Slavery Statement 2018 This statement has been published in accordance with the Modern Slavery Act 2015. It sets out the steps taken by Coca-Cola European Partners plc and its group companies (CCEP), in particular Coca-Cola European Partners Great Britain Limited and Coca-Cola European Partners Services Europe Limited, during the year ending 31 December 2018 to prevent modern slavery and human trafficking in our business and supply chain. Introduction We consider human and workplace rights  as articulated in the United Nations Universal Declaration of Human Rights, and the International Labour Organizations Declaration on Fundamental Principles and Rights at Work  to be inviolable. Respect for human rights is fundamental to the sustainability of CCEP and the communities in which we operate. We are committed to ensuring that all people are treated with dignity and respect. We support the 10 principles of the UN Global Compact, and take a proactive approach to respecting these 

#### Comments
- Exactly what's expected, gets the text from a pdf
- Doesn't retain structure

##### - PyPDF2

In [224]:
import PyPDF2 
import re
import pandas as pd

In [206]:
pdffile = open('coke_1.pdf', 'rb')
pdfreader = PyPDF2.PdfFileReader(pdffile)
# first_page = pdfreader.getPage(0).extractText()
# first_page = first_page.split('\n')

In [327]:
num_pages = pdfreader.getNumPages()
#Get it in text format and split by \n
pages = [pdfreader.getPage(i).extractText().split('\n') for i in range(num_pages)]

In [521]:
#Remove !!'s around the titles'
#Make into a dataframe for easier manipulation
pattern = '(![ ]*)|[   ][A-Z]'
r = re.compile(pattern)
titles = []
for page_num, page in enumerate(pages):
    for line_num, line in enumerate(page):
        if r.match(line):
            if len(line.split()) < 8:
                line = re.sub(r'!|#|"|$|&|%', '', line)
                line = line.strip()
                titles.append((page_num, line_num,line))
titles = pd.DataFrame(titles, columns=['page_number', 'line_number', 'title'])

In [522]:
titles

,page_number,line_number,title
0,0,0,Modern Slavery Statement 2018
1,0,7,Introduction
2,0,24,We have a zero-tolerance approach to Modern
3,1,0,Our business
4,1,33,Child labour
5,1,34,Forced labour
6,1,35,Freedom of association and collective bargaining
7,1,36,Abuse of labour
8,1,37,Discrimination
9,1,38,Wages and benefits


In [525]:
#Making a dictionary out of the titles
#Titles are keys and the text are the values
segments = {}
for i in range(1,len(titles)):
    if titles['page_number'][i-1] == titles['page_number'][i]:
        segments[titles['title'][i-1]] = pages[titles['page_number'][i-1]][titles['line_number'][i-1]+1:titles['line_number'][i]]
        continue
    segments[titles['title'][i-1]] = pages[titles['page_number'][i-1]][titles['line_number'][i-1]+1:]

### Comments

- PyPDF is by far the best

- The data is now in a format where it's sorted by title
- No data is missing as the text in between each title is saved under the previous title and the last title contains the remaining text

##### Data is saved as a dictionary (JSON). NoSQL might be the best database for it.

In [526]:
segments

{'Modern Slavery Statement 2018': [' !This statement has been published in accordance with the Modern Slavery Act 2015.  ',
  'It sets out the steps taken by Coca-Cola European Partners plc and its group ',
  'companies (CCEP), in particular Coca-Cola European Partners Great Britain Limited ',
  'and Coca-Cola European Partners Services Europe Limited, during the year ending 31 ',
  'December 2018 to prevent modern slavery and human trafficking in our business and ',
  'supply chain.'],
 'Introduction': [' We consider human and workplace rights Ð as ',
  'articulated in the United NationsÕ Universal ',
  'Declaration of Human Rights, and the International ',
  'Labour OrganizationÕs Declaration on Fundamental ',
  'Principles and Rights at Work Ð to be inviolable.  ',
  'Respect for human rights is fundamental to the ',
  'sustainability of CCEP and the communities in which ',
  'we operate. We are committed to ensuring that all ',
  'people are treated with dignity and respect.   ',
 

In [527]:
segments.keys()

dict_keys(['Modern Slavery Statement 2018', 'Introduction', 'We have a zero-tolerance approach to Modern', 'Our business', 'Child labour', 'Forced labour', 'Freedom of association and collective bargaining', 'Abuse of labour', 'Discrimination', 'Wages and benefits', 'Work hours and overtime', 'Health and safety', 'Environment', 'Demonstration of compliance', 'Eliminate discrimination', 'Work hours and wages', 'Safe and healthy workplace', 'Community and traditional rights', '$Policies', 'Human Rights Policy', 'Code of Conduct (CoC)', 'Supplier Guiding Principles (SGPs)'])